**GATK Tutorial Notebook Demo**

This notebook illustrates key features of notebooks that we use in GATK workshops to explore how the tools work. The latest versions of the full GATK workshop tutorials are available in the [Terra Showcase](https://app.terra.bio/#library/showcase), in workspaces that are named following the convention "GATKTutorials-Topic-MonthYear".

## Setup
At the beginning we usually do some setup work to make it easier to move data around and call on the tools we need later on. The setup section can often be copied from notebook to notebook, and the Terra team is looking into some options to formalize this into templates to make it even easier. No one wants to have to look up how to do this stuff; we all want to zip straight through to the science... But bear with me for a minute, we'll get there very soon. 

### Adjust the runtime configuration
*If you already followed this instruction in the workspace Dashboard, you can skip this.*

Don't worry, this is pretty straightforward. We want to restart our notebook runtime with a startup script to give it some extra functionality that doesn't come preloaded. Click on the gear icon in the upper right of the window (in the grey "Notebook Runtime" box) and set the values in the small form that pops up as specified below (some may already be set correctly):

| Runtime Parameter | Value |
| ------ | ------ |
| CPU | 4 |
| Disk size | 50 GB |
| Memory | 15 GB |
| Startup Script | `gs://gatk-tutorials/scripts/install_gatk_4110_with_igv.sh` |

Click the "Update" button when you are done, and Terra will begin to create a new runtime with your settings. When it is finished, it will pop up a message asking you to apply the new settings. In the meantime, you can continue with the setup instructions below. 

### Check your kernel
The "kernel" of the notebook is the computational engine that actually executes the code in the notebook. For this particular notebook, we're using a Python 3 kernel so we can execute GATK commands using Python Magic (actual technical term, I swear). That allows us to run terminal commands within the Python environment simply by using the `!` prefix. In the upper right corner of the notebook, just under the Notebook Runtime box, it should say `Python3`. If this notebook isn't running a Python 3 kernel, you can switch it by navigating to the Kernel menu in the Jupyter menu bar and selecting `Change kernel`.

### Grab the workspace bucket identifier
"Bucket" is the name Google gives to the highest level of data storage container, and every workspace in Terra has a dedicated bucket attached to it. The local storage associated with our notebook is temporary, so when we produce outputs that we want to keep, we copy them to the workspace bucket. 

In [ ]:
# Tell Python to look up the relevant environmental variable
import os
BUCKET = os.environ['WORKSPACE_BUCKET']

# Print out the result to make sure what we got makes sense
BUCKET

When you run the cell above, it outputs the path to where our workspace bucket lives in Google Cloud Storage (GCS). It should look something like this: `'gs://fc-49ee40bd-de62-4b05-89c0-4646d7f325e1'`.

### Review the tutorial data
The data bundle we're using here is a set of pre-processsed BAM files from the mother of the CEU trio, NA12878. We're also providing genome reference files, but note that this is a specially prepared reference that only contains the Human chromosome 20 (from the b37 assembly). We use these data snippets and truncated reference in teaching and testing because they are very ssmall and easier to transfer if needed.

The bundle files live in GCS in this location: `gs://gatk-tutorials/notebook-demo/data`. We can list the contents of the tutorial bundle with `gsutil`, a utility toolkit for uploading and managing data in GCS. In both the filepath and the toolkit name, `gs` stands for G(oogle Cloud) S(torage). Why is it sometimes `gs` and sometimes `GCS`? No one knows; it's one of the great remaining mysteries of the universe. 

In [ ]:
! gsutil ls -r gs://gatk-tutorials/notebook-demo/data

You see that for the `mother` sample there are three types of BAMs; that's because we're including a whole genome sequencing (WGS) example, an exome example, and a bulk RNAseq example. In the next section we'll compare the differences by viewing them in IGV.

NOTE: If we wanted to copy files from their permanent location in GCS to the local disk of our notebook runtime under `/home/jupyter-user/`, e.g. to operate on those files with tools that are not capable of streaming directly from GCS, we would use `gsutil` again with the following command. If we want to copy something from the runtime storage to the bucket, ssame command, just flip the order of the paths.  

`! gsutil -m cp -r gs://gatk-tutorials/notebook-demo/data/bams /home/jupyter-user/`

Finally (for now), let's create a directory where we'll put output files.

In [ ]:
! mkdir -p /home/jupyter-user/sandbox/

## Data exploration

Now that our notebook is all set up and ready to roll, let's load some data into IGV. Specifically, we're going to load the three `mother` BAM files for comparison. But first we need to get IGV up and running. There are three . ways to work with IGV in the cloud, but here we're going to focus on the one that involves embedding an IGV window in the notebook.

### Set up IGV

The startup script that we ran earlier contained some installation instructions that ensure we have what we need to run IGV in the notebook. As a result we just need to do a one-time import to activate the `igv` Python package, then after that it's just a matter of creating an IGV window wherever we want one. The only parameter we absolutely have to provide is the genome reference; everything else is optional. Though it is really convenient to preset some coordinates to zoom in on -- or you could provide a gene name instead. Note that the name we give the browser (here, `IGV_Explore`) is completely arbitrary.

In [ ]:
# First invocation: import the IGV Python package
import igv

# Create an interactive IGV browser with genome reference and coordinates specified.
IGV_Explore = igv.Browser(
    {"genome": "hg19",
     "locus": "chr20:10,025,584-10,036,143"
    }
)

# Tell Python to display it below
IGV_Explore.show()

When you run this the first time you should see an embedded IGV window that includes the reference genome and a RefSeq gene track, but no actual data tracks. So now let's load in some samples.

### Load data into IGV

For each track that we want to load, we need to provide this same set of metadata: a name for the track, the path to where the file lives in GCS, the format, and whether to look for an index. The index bit is tricky because of reasons we won't go into here, so for now just leave it to `False` in all cases. 

In [ ]:
IGV_Explore.load_track(
    {
        "name": "Mother WGS",
        "url": "gs://gatk-tutorials/notebook-demo/data/bams/mother.bam",
        "format": "bam",
        "indexed": False
    })

In [ ]:
IGV_Explore.load_track(
    {
        "name": "Mother Exome",
        "url": "gs://gatk-tutorials/notebook-demo/data/bams/motherNEX.bam",
        "format": "bam",
        "indexed": False
    })

In [ ]:
IGV_Explore.load_track(
    {
        "name": "Mother RNAseq",
        "url": "gs://gatk-tutorials/notebook-demo/data/bams/motherRnaseq.bam",
        "format": "bam",
        "indexed": False
    })

Once you've run all three cells above and each of them returned `'OK'` as a result, scroll up and you'll see that you now have three data tracks in your IGV browser: the WGS, Exome and bulk RNAseq versions of the Mother sample, respectively. The grey bars are individual reads, and the spots of colors are missmatches relative to the reference sequence (most mismatches are technical artifacts, but some may corresspond to real variants). 

The coverage profiles (how much read depth there is at any given spot) are highly typical of those datatypes: the WGS is like a distant mountain range; the Exome is like a series a volcanic islands scattered in the ocean; the RNAseq is a lot like the Exome but with cliffs that correspond to splicing sites, and blue lines that connect exons across intronic regions. If you had never seen data like this before, rejoice -- you are now able to distinguish WGS, Exome and bulk RNAseq data on sight!

## Variant calling

Ok, looking at sequence data is cool but it doesn't exactly move the needle in terms of identifying variants. Sure, IGV gives you a pretty reasonable back of the envelope estimation of what might look like a variant based on the relative proportions of alleles at any given site, but can you really trust that? Can you? (spoiler: if you could, there wouldn't be a reason for GATK to exist, would there)

So let's run HaplotypeCaller, our flagship caller for germline short variants, i.e. SNPs and Indels. 

### Run HaplotypeCaller in default mode

To keep things simple, let's start out by running the tool in normal mode on a single sample, the Mother WGS. 

In [ ]:
! gatk HaplotypeCaller \
    -R gs://gatk-tutorials/notebook-demo/data/ref/ref.fasta \
    -I gs://gatk-tutorials/notebook-demo/data/bams/mother.bam \
    -O /home/jupyter-user/sandbox/motherHC.vcf \
    -L 20:10,000,000-10,200,000

That sshould take a minute or two to complete... All good? Hey, that was the first GATK command in the notebook, nicely done! We can list the sandbox contents to confirm that the command worked and the VCF of variant calls was created as expected.

In [ ]:
! ls /home/jupyter-user/sandbox/

So now we have our output VCF, we want to open it up in IGV to do a visual check. 

### Load the data (BAM and VCF) into IGV

Bit of a caveat here: IGV needs the file to be in GCS, but right now it's only in our notebook's local storage, so we need to copy it to the workspace bucket to make it visible to IGV.

In [ ]:
! gsutil cp -a public-read /home/jupyter-user/sandbox/* $BUCKET/sandbox

Done? Great, now let's make a new IGV browser right here. We're going to look at a different region because we hand-picked a call that we want to inspect.

In [ ]:
# Create an interactive IGV browser with genome reference and coordinates specified.
IGV_InspectCalls = igv.Browser(
    {"genome": "hg19",
     "locus": "chr20:10,002,294-10,002,623"
    }
)

# Tell Python to display it below
IGV_InspectCalls.show()

In [ ]:
# Load the variant track
IGV_InspectCalls.load_track(
    {
        "name": "Mother HC variants",
        "url": BUCKET + "/sandbox/motherHC.vcf",
        "format": "vcf",
        "indexed": False
    })

In [ ]:
# Load the original sequence data track
IGV_InspectCalls.load_track(
    {
        "name": "Mother WGS",
        "url": "gs://gatk-tutorials/notebook-demo/data/bams/mother.bam",
        "format": "bam",
        "indexed": False
    })

Oh snap -- we have two variant calls, and one makes a ton of sense given what we see in the reads, but the other one not so much. Can you guess which is which? 

### Troubleshooting a weird call

We see that HaplotypeCaller called a homozygous variant insertion of three T bases... How is this possible when so few reads seem to support an insertion at this position? Well, your first reflex when you encounter indel-related weirdness should be to turn on the display of soft-clips, which IGV turns off by default. 

To turn on soft clips, select the gear symbol to the right of the track and select "Show soft clips". You should see the whole area light up in the bright glare of a million mismatches. Uh-oh. Time to generate a "bamout" to see what HaplotypeCaller was thinking...

In [ ]:
! gatk HaplotypeCaller \
    -R gs://gatk-tutorials/notebook-demo/data/ref/ref.fasta \
    -I gs://gatk-tutorials/notebook-demo/data/bams/mother.bam \
    -O /home/jupyter-user/sandbox/motherHCdebug.vcf \
    -bamout /home/jupyter-user/sandbox/motherHCdebug.bam \
    -L 20:10,002,000-10,003,000

That should run very quickly since we've really narrowed down the region we're processing. And once again, we copy the output to the bucket so it will be visible to IGV. We just re-run the same "copy all the sandbox contents" command each time because we're lazy and the transfer within Google's network is so fast it doasn't really matter, but you could identify specific files if you wanted to.

In [ ]:
! gsutil cp -a public-read /home/jupyter-user/sandbox/* $BUCKET/sandbox

In [ ]:
# Load the bamout data track
IGV_InspectCalls.load_track(
    {
        "name": "Mother HC bamout",
        "url": BUCKET + "/sandbox/motherHCdebug.bam",
        "height": 500,
        "format": "bam",
        "indexed": False
    })

Now scroll back up and take another look. What do you think? Does HaplotypeCaller know what it's doing or what?


## Next steps

There's a lot more to the interpretation of what's going on there than we have time for in our little demo. The full germline short variant discovery tutorial covers how to investigate this case in more detail using IGV and other tools, and how to go on from here to run joint calling on cohorts of any size. It is complemented by companion notebooks that show you how to run different types of filtering approaches (including deep learning with GATK CNN) and how to make plots of annotation distributions to get a better sense of what that all means.

Be sure to check out the full tutorials in the [showcase](https://app.terra.bio/#library/showcase) for much more detail about what's going on at the various steps, and to see all the other use cases currently covered by tutorial notebooks, which includes Mutect2 and the somatic copy number (CNV) analysis pipelines (and germlin CNVs are coming soon). You may also want to check out the [Terra Notebooks Playground](https://app.terra.bio/#workspaces/help-gatk/Terra%20Notebooks%20Playground) for cookbook-style examples of how to do common tasks with notebooks in Terra.